# Checking CREODIAS

Do they have everything we need for the Planetary Computer?
First, what do we have in the Planetary Computer?

In [9]:
from IPython.display import display
from pystac_client import Client
from rich.table import Table

client = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
collections = [
    c
    for c in client.get_collections()
    if c.id.startswith("sentinel-3")
    or c.id.startswith("sentinel-5p")
    or c.id.startswith("sentinel-1")
]
table = Table("id", "description")
for collection in collections:
    table.add_row(collection.id, collection.description[0:100] + "...")
display(table)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ id                               ┃ description                                                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sentinel-1-rtc                   │ The [Sentinel-1](https://sentinel.esa.int/web/sentinel/missions/sentinel-1)  │
│                                  │ mission is a constellati...                                                  │
│ sentinel-1-grd                   │ The [Sentinel-1](https://sentinel.esa.int/web/sentinel/missions/sentinel-1)  │
│                                  │ mission is a constellati...                                                  │
│ sentinel-5p-l2-netcdf            │ The Copernicus [Sentinel-5                                                   │
│                                  │ Precursor](https://sentinels.copernicus.eu/web/sentinel/missions/sentinel... │
│ sentinel-3-olci-wfr-l2-netcdf    │ This Collection provides Sentinel-3 Full Resolution [OLCI Level-2 Water]     │
│                                  │ products containin...                                                        │
│ sentinel-3-synergy-aod-l2-netcdf │ This Collection provides the Sentinel-3 [Synergy Level-2 Aerosol Optical     │
│                                  │ Depth](https://sentinels.co...                                               │
│ sentinel-3-synergy-v10-l2-netcdf │ This Collection provides the Sentinel-3 [Synergy Level-2 10-Day Surface      │
│                                  │ Reflectance and NDVI](https:...                                              │
│ sentinel-3-olci-lfr-l2-netcdf    │ This collection provides Sentinel-3 Full Resolution [OLCI Level-2 Land]      │
│                                  │ products containing...                                                       │
│ sentinel-3-sral-lan-l2-netcdf    │ This Collection provides Sentinel-3 [SRAL Level-2 Land                       │
│                                  │ Altimetry](https://sentinel.esa.int/web/senti...                             │
│ sentinel-3-slstr-lst-l2-netcdf   │ This Collection provides Sentinel-3 [SLSTR Level-2 Land Surface              │
│                                  │ Temperature](https://sentinel.esa.in...                                      │
│ sentinel-3-slstr-wst-l2-netcdf   │ This Collection provides Sentinel-3 [SLSTR Level-2 Water Surface             │
│                                  │ Temperature](https://sentinel.esa.i...                                       │
│ sentinel-3-sral-wat-l2-netcdf    │ This Collection provides Sentinel-3 [SRAL Level-2 Ocean                      │
│                                  │ Altimetry](https://sentinel.esa.int/web/sent...                              │
│ sentinel-3-slstr-frp-l2-netcdf   │ This Collection provides Sentinel-3 [SLSTR Level-2 Fire Radiative            │
│                                  │ Power](https://sentinel.esa.int/we...                                        │
│ sentinel-3-synergy-syn-l2-netcdf │ This Collection provides the Sentinel-3 [Synergy Level-2 Land Surface        │
│                                  │ Reflectance and Aerosol](https...                                            │
│ sentinel-3-synergy-vgp-l2-netcdf │ This Collection provides the Sentinel-3 [Synergy Level-2 Top of Atmosphere   │
│                                  │ Reflectance](https://sent...                                                 │
│ sentinel-3-synergy-vg1-l2-netcdf │ This Collection provides the Sentinel-3 [Synergy Level-2 1-Day Surface       │
│                                  │ Reflectance and NDVI](https:/...                                             │
└──────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────┘

## Sentinel 3

Let's start with Sentinel 3.
We want to make sure that every sensor and product type is in the CREODIAS API.

In [10]:
import copy
from requests import Session

SEARCH_URL = "https://datahub.creodias.eu/odata/v1/Products"
session = Session()

query = {
    "$top": 1,
    "$orderby": "PublicationDate asc",
}
table = Table("collection id", "platform", "item name", title="sentinel-3")
for collection in collections:
    if not collection.id.startswith("sentinel-3"):
        continue
    product_type = collection.summaries.get_list("s3:product_type")[0]
    for prefix in ("S3A", "S3B"):
        collection_query = copy.deepcopy(query)
        collection_query["$filter"] = (
            "Collection/Name eq 'SENTINEL-3' and "
            "PublicationDate gt 2023-05-01T00:00:00Z and "
            f"startswith(Name, '{prefix}_{product_type}')"
        )
        response = session.get(SEARCH_URL, params=collection_query)
        value = response.json()["value"]
        if value:
            assert len(value) == 1
            table.add_row(collection.id, prefix, value[0]["Name"])
        else:
            table.add_row(collection.id, prefix, "—")
display(table)

                                                    sentinel-3                                                     
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ collection id                    ┃ platform ┃ item name                                                         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sentinel-3-olci-wfr-l2-netcdf    │ S3A      │ S3A_OL_2_WFR____20230422T020402_20230422T020702_20230422T041202_… │
│ sentinel-3-olci-wfr-l2-netcdf    │ S3B      │ S3B_OL_2_WFR____20230420T173204_20230420T173504_20230422T030049_… │
│ sentinel-3-synergy-aod-l2-netcdf │ S3A      │ S3A_SY_2_AOD____20230429T070010_20230429T074430_20230430T235229_… │
│ sentinel-3-synergy-aod-l2-netcdf │ S3B      │ S3B_SY_2_AOD____20230430T005148_20230430T013607_20230501T022840_… │
│ sentinel-3-synergy-v10-l2-netcdf │ S3A      │ S3A_SY_2_V10____20230421T000000_20230430T235959_20230501T232158_… │
│ sentinel-3-synergy-v10-l2-netcdf │ S3B      │ S3B_SY_2_V10____20230421T000000_20230430T235959_20230502T123616_… │
│ sentinel-3-olci-lfr-l2-netcdf    │ S3A      │ S3A_OL_2_LFR____20230430T222624_20230430T222704_20230501T000027_… │
│ sentinel-3-olci-lfr-l2-netcdf    │ S3B      │ S3B_OL_2_LFR____20230430T060221_20230430T060521_20230430T234003_… │
│ sentinel-3-sral-lan-l2-netcdf    │ S3A      │ S3A_SR_2_LAN____20230430T222851_20230430T223850_20230501T000155_… │
│ sentinel-3-sral-lan-l2-netcdf    │ S3B      │ S3B_SR_2_LAN____20230406T185722_20230406T194751_20230430T235931_… │
│ sentinel-3-slstr-lst-l2-netcdf   │ S3A      │ S3A_SL_2_LST____20230430T224724_20230430T225024_20230430T235946_… │
│ sentinel-3-slstr-lst-l2-netcdf   │ S3B      │ S3B_SL_2_LST____20230430T110218_20230430T110518_20230430T133645_… │
│ sentinel-3-slstr-wst-l2-netcdf   │ S3A      │ S3A_SL_2_WST____20230414T081329_20230414T081629_20230414T092811_… │
│ sentinel-3-slstr-wst-l2-netcdf   │ S3B      │ S3B_SL_2_WST____20230425T204809_20230425T205109_20230425T223036_… │
│ sentinel-3-sral-wat-l2-netcdf    │ S3A      │ S3A_SR_2_WAT____20230425T050158_20230425T050845_20230425T061523_… │
│ sentinel-3-sral-wat-l2-netcdf    │ S3B      │ S3B_SR_2_WAT____20230412T171045_20230412T175909_20230414T092220_… │
│ sentinel-3-slstr-frp-l2-netcdf   │ S3A      │ S3A_SL_2_FRP____20230330T013248_20230330T013748_20230330T040243_… │
│ sentinel-3-slstr-frp-l2-netcdf   │ S3B      │ S3B_SL_2_FRP____20230426T164749_20230426T165249_20230426T184724_… │
│ sentinel-3-synergy-syn-l2-netcdf │ S3A      │ S3A_SY_2_SYN____20230430T132527_20230430T132827_20230430T235138_… │
│ sentinel-3-synergy-syn-l2-netcdf │ S3B      │ S3B_SY_2_SYN____20230430T164416_20230430T164457_20230501T000611_… │
│ sentinel-3-synergy-vgp-l2-netcdf │ S3A      │ S3A_SY_2_VGP____20230429T033813_20230429T042232_20230430T200605_… │
│ sentinel-3-synergy-vgp-l2-netcdf │ S3B      │ S3B_SY_2_VGP____20230430T160037_20230430T164457_20230430T220131_… │
│ sentinel-3-synergy-vg1-l2-netcdf │ S3A      │ S3A_SY_2_VG1____20230429T000000_20230429T235959_20230501T201922_… │
│ sentinel-3-synergy-vg1-l2-netcdf │ S3B      │ S3B_SY_2_VG1____20230430T000000_20230430T235959_20230501T123230_… │
└──────────────────────────────────┴──────────┴───────────────────────────────────────────────────────────────────┘

## Sentinel 5P

Now let's do a similar exercise for the `sentinel-5p` collection.

In [11]:
collection = next(c for c in collections if c.id.startswith("sentinel-5p"))
table = Table("product type", "item name", title="sentinel-5p")
for product_type in collection.summaries.get_list("s5p:product_type"):
    collection_query = copy.deepcopy(query)
    collection_query["$filter"] = (
        "Collection/Name eq 'SENTINEL-5P' and "
        "PublicationDate gt 2023-05-01T00:00:00Z and "
        f"startswith(Name, 'S5P_NRTI_{product_type}')"
    )
    response = session.get(SEARCH_URL, params=collection_query)
    value = response.json()["value"]
    if value:
        assert len(value) == 1
        table.add_row(product_type, value[0]["Name"])
    else:
        table.add_row(product_type, "—")
display(table)

                                               sentinel-5p                                               
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ product type ┃ item name                                                                              ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ L2__AER_AI   │ S5P_NRTI_L2__AER_AI_20230420T233422_20230420T233922_28596_03_020500_20230421T003803.nc │
│ L2__AER_LH   │ S5P_NRTI_L2__AER_LH_20230413T105422_20230413T105922_28489_03_020500_20230413T114625.nc │
│ L2__CH4___   │ —                                                                                      │
│ L2__CLOUD_   │ S5P_NRTI_L2__CLOUD__20230405T020922_20230405T021422_28370_03_020401_20230405T034721.nc │
│ L2__CO____   │ S5P_NRTI_L2__CO_____20230413T105922_20230413T110422_28489_03_020500_20230413T114531.nc │
│ L2__HCHO__   │ S5P_NRTI_L2__HCHO___20230410T200422_20230410T200922_28452_03_020401_20230410T205448.nc │
│ L2__NO2___   │ S5P_NRTI_L2__NO2____20230405T094422_20230405T094922_28375_03_020500_20230405T103850.nc │
│ L2__NP_BD3   │ —                                                                                      │
│ L2__NP_BD6   │ —                                                                                      │
│ L2__NP_BD7   │ —                                                                                      │
│ L2__O3_TCL   │ —                                                                                      │
│ L2__O3____   │ S5P_NRTI_L2__O3_____20230413T105422_20230413T105922_28489_03_020401_20230413T114620.nc │
│ L2__SO2___   │ S5P_NRTI_L2__SO2____20230413T023422_20230413T023922_28484_03_020401_20230413T031610.nc │
└──────────────┴────────────────────────────────────────────────────────────────────────────────────────┘

### Missing data

Looks like we're missing:

- `L2__CH4___`
- `L2__NP_BD3`
- `L2__NP_BD6`
- `L2__NP_BD7`
- `L2__O3_TCL`

Let's see if they're `OFFL` instead of `NRTI`.

In [12]:
missing_product_types = [
    "L2__CH4___",
    "L2__NP_BD3",
    "L2__NP_BD6",
    "L2__NP_BD7",
    "L2__O3_TCL",
]
table = Table("product type", "item name", title="sentinel-5P OFFL")
for product_type in missing_product_types:
    collection_query = copy.deepcopy(query)
    collection_query["$filter"] = (
        "Collection/Name eq 'SENTINEL-5P' and "
        "PublicationDate gt 2023-05-01T00:00:00Z and "
        f"startswith(Name, 'S5P_OFFL_{product_type}')"
    )
    response = session.get(SEARCH_URL, params=collection_query)
    value = response.json()["value"]
    if value:
        assert len(value) == 1
        table.add_row(product_type, value[0]["Name"])
    else:
        table.add_row(product_type, "—")
display(table)

                                            sentinel-5P OFFL                                             
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ product type ┃ item name                                                                              ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ L2__CH4___   │ S5P_OFFL_L2__CH4____20230403T095605_20230403T113735_28347_03_020500_20230405T021404.nc │
│ L2__NP_BD3   │ S5P_OFFL_L2__NP_BD3_20230423T171018_20230423T185148_28635_03_010300_20230425T065828.nc │
│ L2__NP_BD6   │ S5P_OFFL_L2__NP_BD6_20230402T032855_20230402T051026_28329_03_010300_20230403T171243.nc │
│ L2__NP_BD7   │ S5P_OFFL_L2__NP_BD7_20230329T113029_20230329T131159_28277_03_010300_20230331T012002.nc │
│ L2__O3_TCL   │ S5P_OFFL_L2__O3_TCL_20230321T110002_20230327T114642_28206_03_020401_20230405T000319.nc │
└──────────────┴────────────────────────────────────────────────────────────────────────────────────────┘

## Sentinel 1

In [13]:
table = Table("platform", "item name", title="sentinel-1-grd")
for prefix in ("S1A", "S1B"):
    collection_query = copy.deepcopy(query)
    collection_query["$filter"] = (
        "Collection/Name eq 'SENTINEL-1' and "
        "PublicationDate gt 2023-05-01T00:00:00Z and "
        f"startswith(Name, '{prefix}')"
    )
    response = session.get(SEARCH_URL, params=collection_query)
    value = response.json()["value"]
    if value:
        assert len(value) == 1
        table.add_row(prefix, value[0]["Name"])
    else:
        table.add_row(prefix, "—")
display(table)

                                      sentinel-1-grd                                       
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ platform ┃ item name                                                                    ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ S1A      │ S1A_EW_GRDM_1SSH_20151008T122047_20151008T122147_008061_00B4AE_5A07_COG.SAFE │
│ S1B      │ S1B_EW_GRDM_1SSH_20160926T000338_20160926T000438_002232_003C0F_1F0B_COG.SAFE │
└──────────┴──────────────────────────────────────────────────────────────────────────────┘

## Conclusion

We're good!
We've got them all, or at least SOME data exists for all PC products since the start of May 2023.
We'll want to look at the `OFFL` vs `NRTI` vs `RPRO` for Sentinel-5P, but we _should_ be able to find what we need.